In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import tensorflow
import tensorflow as tf
print(tf.__version__)

import keras
import keras.backend as K
from keras.models import Model
from keras.layers import Input, Dense, Conv2D, Conv3D, DepthwiseConv2D, SeparableConv2D, Conv3DTranspose
from keras.layers import Flatten, MaxPool2D, AvgPool2D, GlobalAvgPool2D, UpSampling2D, BatchNormalization
from keras.layers import Concatenate, Add, Dropout, ReLU, Lambda, Activation, LeakyReLU, PReLU



from time import time
import numpy as np

from keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping

import warnings
warnings.filterwarnings('ignore')

2.9.3


In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train=ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True,validation_split = 0.2)
train_data=train.flow_from_directory(directory = 'dataset/TRAIN',target_size=(224,224),
                                     batch_size=32,class_mode='categorical')

Found 2000 images belonging to 5 classes.


In [4]:
test=ImageDataGenerator(rescale=1./255)
test_data=test.flow_from_directory(directory = 'dataset/TEST',target_size=(224,224),
                                   batch_size=32,class_mode='categorical')

Found 2000 images belonging to 5 classes.


In [5]:
def shufflenet(input_shape, n_classes, g=8):
  channels = 384, 769, 1536
  repetitions = 3, 7, 3
  
  def ch_shuffle(x, g):
#     1 2 3 4 5 6 7 8 9 -reshape-> 1 2 3 -permute dims-> 1 4 7 -reshape-> 1 4 7 2 5 8 3 6 9
#                                  4 5 6                 2 5 8
#                                  7 8 9                 3 6 9
    
    _, w, h, ch = K.int_shape(x)
    ch_g = ch // g
    
    def shuffle_op(x):
      x = K.reshape(x, [-1, w, h, ch_g, g])
      x = K.permute_dimensions(x, [0, 1, 2, 4, 3])
      x = K.reshape(x, [-1, w, h, ch])
      return x
    
    x = Lambda(shuffle_op)(x)
    return x
    
  
  def gconv(tensor, ch, g):
    _, _, _, in_ch = K.int_shape(tensor)
    ch_g = in_ch // g
    out_ch = ch // g
    group = []
    
    for i in range(g):
#       x = tensor[:, :, :, i*ch_g:(i+1)*ch_g]
      x = Lambda(lambda x: x[:, :, :, i*ch_g: (i+1)*ch_g])(tensor)
      x = Conv2D(out_ch, 1)(x)
      group.append(x)
    
    x = Concatenate()(group)
    return x
  
  
  def shufflenet_block(tensor, ch, s, g):
    x = gconv(tensor, ch // 4, g)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = ch_shuffle(x, g)
    x = DepthwiseConv2D(3, strides=s, padding='same')(x)
    x = BatchNormalization()(x)
    x = gconv(x, ch if s==1 else ch-K.int_shape(tensor)[-1], g)
    x = BatchNormalization()(x)
    
    if s == 1:
      x = Add()([tensor, x])
    else:
      avg = AvgPool2D(3, strides=2, padding='same')(tensor)
      x = Concatenate()([avg, x])
    
    output = ReLU()(x)
    return output
  
  
  def stage(x, ch, r, g):
    x = shufflenet_block(x, ch, 2, g)
    
    for i in range(r):
      x = shufflenet_block(x, ch, 1, g)
      
    return x
  
  input = Input(input_shape)
  
  x = Conv2D(24, 3, strides=2, padding='same')(input)
  x = BatchNormalization()(x)
  x = ReLU()(x)
  x = MaxPool2D(3, strides=2, padding='same')(x)
  
  for ch, r in zip(channels, repetitions):
    x = stage(x, ch, r, g)
    
  x = GlobalAvgPool2D()(x)
  output = Dense(n_classes, activation='softmax')(x)
  
  model = Model(input, output)
  model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy',tensorflow.keras.metrics.Precision()])
  return model
     

input_shape = 224, 224, 3
n_classes = 5

K.clear_session()
model = shufflenet(input_shape, n_classes)
model.summary() 

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 112, 112, 24  672         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 112, 112, 24  96         ['conv2d[0][0]']                 
 alization)                     )                                                             

 lambda_83 (Lambda)             (None, 14, 14, 24)   0           ['batch_normalization_14[0][0]'] 
                                                                                                  
 lambda_84 (Lambda)             (None, 14, 14, 24)   0           ['batch_normalization_14[0][0]'] 
                                                                                                  
 conv2d_73 (Conv2D)             (None, 14, 14, 48)   1200        ['lambda_77[0][0]']              
                                                                                                  
 conv2d_74 (Conv2D)             (None, 14, 14, 48)   1200        ['lambda_78[0][0]']              
                                                                                                  
 conv2d_75 (Conv2D)             (None, 14, 14, 48)   1200        ['lambda_79[0][0]']              
                                                                                                  
 conv2d_76

                                                                                                  
 re_lu_11 (ReLU)                (None, 14, 14, 192)  0           ['batch_normalization_16[0][0]'] 
                                                                                                  
 lambda_93 (Lambda)             (None, 14, 14, 192)  0           ['re_lu_11[0][0]']               
                                                                                                  
 depthwise_conv2d_5 (DepthwiseC  (None, 14, 14, 192)  1920       ['lambda_93[0][0]']              
 onv2D)                                                                                           
                                                                                                  
 batch_normalization_17 (BatchN  (None, 14, 14, 192)  768        ['depthwise_conv2d_5[0][0]']     
 ormalization)                                                                                    
          

                                                                                                  
 conv2d_101 (Conv2D)            (None, 14, 14, 24)   2328        ['lambda_106[0][0]']             
                                                                                                  
 conv2d_102 (Conv2D)            (None, 14, 14, 24)   2328        ['lambda_107[0][0]']             
                                                                                                  
 conv2d_103 (Conv2D)            (None, 14, 14, 24)   2328        ['lambda_108[0][0]']             
                                                                                                  
 conv2d_104 (Conv2D)            (None, 14, 14, 24)   2328        ['lambda_109[0][0]']             
                                                                                                  
 concatenate_14 (Concatenate)   (None, 14, 14, 192)  0           ['conv2d_97[0][0]',              
          

                                                                                                  
 lambda_121 (Lambda)            (None, 14, 14, 96)   0           ['re_lu_14[0][0]']               
                                                                                                  
 lambda_122 (Lambda)            (None, 14, 14, 96)   0           ['re_lu_14[0][0]']               
                                                                                                  
 lambda_123 (Lambda)            (None, 14, 14, 96)   0           ['re_lu_14[0][0]']               
                                                                                                  
 lambda_124 (Lambda)            (None, 14, 14, 96)   0           ['re_lu_14[0][0]']               
                                                                                                  
 lambda_125 (Lambda)            (None, 14, 14, 96)   0           ['re_lu_14[0][0]']               
          

...

                                                                                                  
 conv2d_201 (Conv2D)            (None, 7, 7, 96)     4704        ['lambda_213[0][0]']             
                                                                                                  
 conv2d_202 (Conv2D)            (None, 7, 7, 96)     4704        ['lambda_214[0][0]']             
                                                                                                  
 conv2d_203 (Conv2D)            (None, 7, 7, 96)     4704        ['lambda_215[0][0]']             
                                                                                                  
 conv2d_204 (Conv2D)            (None, 7, 7, 96)     4704        ['lambda_216[0][0]']             
                                                                                                  
 conv2d_205 (Conv2D)            (None, 7, 7, 96)     4704        ['lambda_217[0][0]']             
          

 lambda_229 (Lambda)            (None, 7, 7, 384)    0           ['re_lu_27[0][0]']               
                                                                                                  
 depthwise_conv2d_13 (Depthwise  (None, 7, 7, 384)   3840        ['lambda_229[0][0]']             
 Conv2D)                                                                                          
                                                                                                  
 batch_normalization_41 (BatchN  (None, 7, 7, 384)   1536        ['depthwise_conv2d_13[0][0]']    
 ormalization)                                                                                    
                                                                                                  
 lambda_230 (Lambda)            (None, 7, 7, 48)     0           ['batch_normalization_41[0][0]'] 
                                                                                                  
 lambda_23

 conv2d_230 (Conv2D)            (None, 7, 7, 48)     9264        ['lambda_243[0][0]']             
                                                                                                  
 conv2d_231 (Conv2D)            (None, 7, 7, 48)     9264        ['lambda_244[0][0]']             
                                                                                                  
 conv2d_232 (Conv2D)            (None, 7, 7, 48)     9264        ['lambda_245[0][0]']             
                                                                                                  
 concatenate_31 (Concatenate)   (None, 7, 7, 384)    0           ['conv2d_225[0][0]',             
                                                                  'conv2d_226[0][0]',             
                                                                  'conv2d_227[0][0]',             
                                                                  'conv2d_228[0][0]',             
          

 lambda_258 (Lambda)            (None, 7, 7, 192)    0           ['re_lu_30[0][0]']               
                                                                                                  
 lambda_259 (Lambda)            (None, 7, 7, 192)    0           ['re_lu_30[0][0]']               
                                                                                                  
 lambda_260 (Lambda)            (None, 7, 7, 192)    0           ['re_lu_30[0][0]']               
                                                                                                  
 lambda_261 (Lambda)            (None, 7, 7, 192)    0           ['re_lu_30[0][0]']               
                                                                                                  
 lambda_262 (Lambda)            (None, 7, 7, 192)    0           ['re_lu_30[0][0]']               
                                                                                                  
 conv2d_24

                                                                  'conv2d_252[0][0]',             
                                                                  'conv2d_253[0][0]',             
                                                                  'conv2d_254[0][0]',             
                                                                  'conv2d_255[0][0]',             
                                                                  'conv2d_256[0][0]']             
                                                                                                  
 batch_normalization_48 (BatchN  (None, 7, 7, 1536)  6144        ['concatenate_34[0][0]']         
 ormalization)                                                                                    
                                                                                                  
 add_12 (Add)                   (None, 7, 7, 1536)   0           ['re_lu_30[0][0]',               
          

In [6]:
model_path = "model.h5"

from keras.callbacks import ModelCheckpoint

M = ModelCheckpoint(model_path, monitor='accuracy', verbose=1, save_best_only=True)

In [7]:
epochs = 100
batch_size = 32

In [ ]:
#### Fitting the model
history = model.fit(
           train_data, steps_per_epoch=train_data.samples // batch_size, 
           epochs=epochs, 
           validation_data=test_data,validation_steps=test_data.samples // batch_size,
           callbacks=[M])

Epoch 1/100
62/62 [==============================] - ETA: 0s - loss: 1.4958 - accuracy: 0.3425 - precision: 0.3851 
Epoch 1: accuracy improved from -inf to 0.34248, saving model to model.h5
62/62 [==============================] - 953s 15s/step - loss: 1.4958 - accuracy: 0.3425 - precision: 0.3851 - val_loss: 1.6227 - val_accuracy: 0.2102 - val_precision: 0.0000e+00
Epoch 2/100
62/62 [==============================] - ETA: 0s - loss: 1.2504 - accuracy: 0.4314 - precision: 0.5046 
Epoch 2: accuracy improved from 0.34248 to 0.43140, saving model to model.h5
62/62 [==============================] - 862s 14s/step - loss: 1.2504 - accuracy: 0.4314 - precision: 0.5046 - val_loss: 1.6434 - val_accuracy: 0.2016 - val_precision: 0.0000e+00
Epoch 3/100
62/62 [==============================] - ETA: 0s - loss: 1.3516 - accuracy: 0.4029 - precision: 0.4311 

In [ ]:
history.history.keys()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.figure(figsize=(20, 8))
plt.plot(history.history['accuracy'])

for i in range(epochs):
    if i%5 == 0:
        plt.annotate(np.round(history.history['accuracy'][i]*100,2),xy=(i,history.history['accuracy'][i]))

plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.show()

In [ ]:
plt.figure(figsize=(20, 8))
plt.plot(history.history['loss'])

for i in range(epochs):
    if i%5 == 0:
        plt.annotate(np.round(history.history['loss'][i]*100,2),xy=(i,history.history['loss'][i]))

plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.show()